In [ ]:
import os
import cv2
import requests
from os import path
import numpy as np
from tqdm import tqdm
from PIL import Image

from rectangle import Rectangle
import helpers
from helpers import *

kindle_ar = (1430,1080)

In [ ]:
if not path.exists(f'dump'):
    os.mkdir('dump')

name = 'Onepunch-Man'
if not path.exists(f'dump/{name}'):
    os.mkdir(f'dump/{name}')

for chap in tqdm(range(1,2)):
    for i in range(1,50):
        # helpers.debug = True
        # print(f'{i:03d}')
        try:
            raw_img = requests.get(f'https://official-ongoing-2.gamindustri.us/manga/{name}/{chap:04d}-{i:03d}.png', stream=True).raw
            img = np.array(Image.open(raw_img))
        except:
            break

        if not path.exists(f'dump/{name}/{chap:04d}'):
            os.mkdir(f'dump/{name}/{chap:04d}')
    
        # img = cv2.imread('0012-002.png')
        # plot_crops(img, [])
        contours = get_contours(img)
        crops = get_crops(img, contours)
        crops = merge_overlapping_crops(crops)
        crops = remove_small_crops(img, crops)
        # debug = True
        # plot_frames(img, crops, True)
        edge_graph = build_crops_edge_graph(crops, img.shape)
        # print(*[(i,j) for i,j in enumerate(edge_graph)], sep='\n')
        sorted_crops = sort_edge_graph(crops, edge_graph)
        # plot_frames(img, sorted_crops,True)
        # plot_crops(img, sorted_crops)
        idx = 0
        for c in sorted_crops:
            gray_crop = cv2.cvtColor(img[c.y:c.z, c.x:c.w], cv2.COLOR_BGR2GRAY)
            # print(gray_crop.shape[1],gray_crop.shape[0],gray_crop.shape[1]/gray_crop.shape[0])
            if gray_crop.shape[1]/gray_crop.shape[0]>0.80 or c.area < img.shape[0]*img.shape[1]*0.5:
                gray_crop = [np.rot90(gray_crop)]
            else:
                gray_crop = [np.rot90(h) for h in np.array_split(gray_crop, 2)]
                
            for gc in gray_crop:
                gc = resizeAndPad(gc, kindle_ar, 255)
                # plot_crops(gc, [])
                # gc = np.rot90(gc,3)
                cv2.imwrite(f"dump/{name}/{chap:04d}/{i:03d}-{idx:03d}.jpg", gc, [int(cv2.IMWRITE_JPEG_QUALITY), 20])
                idx+=1
        # print('-----------------------------------------------------')

In [ ]:
os.system(f"cd dump; zip -FSr {name} {name} -x '*/\.*'")